## Database parsing

In [1]:
import sqlite3

In [2]:
# create a sample databse

conn = sqlite3.connect('../data/database/company.db')
cursor = conn.cursor()

In [3]:
# create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    role TEXT NOT NULL,
    department TEXT NOT NULL,
    salary REAL NOT NULL
)
''')

In [4]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS projects (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    budget REAL NOT NULL,
    lead_id INTEGER
)
''')

In [5]:
# Create sample data

employees = [
    (1, 'Alice Johnson', 'Software Engineer', 'Development', 90000),
    (2, 'Bob Smith', 'Data Scientist', 'Data Science', 95000),
    (3, 'Charlie Brown', 'Product Manager', 'Product', 105000),
    (4, 'Diana Prince', 'UX Designer', 'Design', 85000)
]

projects = [
    (1, 'Project Alpha', 150000, 3),
    (2, 'Project Beta', 200000, 1),
    (3, 'Project Gamma', 120000, 2),
    (4, 'Project Delta', 180000, 4)
]


In [6]:
# Insert sample data

# cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?, ?)', employees)
# cursor.executemany('INSERT INTO projects VALUES (?, ?, ?, ?)', projects)

In [7]:
# same queries

cursor.execute('SELECT * FROM employees')
employees_data = cursor.fetchall()

employees_data

[(1, 'Alice Johnson', 'Software Engineer', 'Development', 90000.0),
 (2, 'Bob Smith', 'Data Scientist', 'Data Science', 95000.0),
 (3, 'Charlie Brown', 'Product Manager', 'Product', 105000.0),
 (4, 'Diana Prince', 'UX Designer', 'Design', 85000.0)]

In [8]:
conn.commit()
conn.close()

In [9]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [12]:
db = SQLDatabase.from_uri("sqlite:///../data/database/company.db")

## Get database info

print(f"Database Name: {db.get_usable_table_names()}")
print('\n\n---TABLE DDL---')
print(f"Database Info: {db.get_table_info()}")


Database Name: ['employees', 'projects']


---TABLE DDL---
Database Info: 
CREATE TABLE employees (
	id INTEGER, 
	name TEXT NOT NULL, 
	role TEXT NOT NULL, 
	department TEXT NOT NULL, 
	salary REAL NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	Alice Johnson	Software Engineer	Development	90000.0
2	Bob Smith	Data Scientist	Data Science	95000.0
3	Charlie Brown	Product Manager	Product	105000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT NOT NULL, 
	budget REAL NOT NULL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	budget	lead_id
1	Project Alpha	150000.0	3
2	Project Beta	200000.0	1
3	Project Gamma	120000.0	2
*/


In [6]:
from typing import List
from langchain_core.documents import Document
import sqlite3

def db_table_summary_docs(db_path: str) -> List[Document]:
    '''Connects to a SQLite DB, extracts table info, and creates summary Documents. Also creates a single joined employee-project doc.'''
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    documents = []
    # Table summaries
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [row[0] for row in cursor.fetchall()]
    for table in tables:
        cursor.execute(f"PRAGMA table_info({table})")
        columns = [row[1] for row in cursor.fetchall()]
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        total_records = cursor.fetchone()[0]
        cursor.execute(f"SELECT * FROM {table} LIMIT 1")
        sample_row = cursor.fetchone()
        sample_dict = dict(zip(columns, sample_row)) if sample_row else {}
        sample_preview = dict(list(sample_dict.items())[:5])
        content = (
            f"Table: {table}\n"
            f"Columns: {columns}\n"
            f"Total Records: {total_records}\n"
            f"Sample Record (first 5 fields): {sample_preview}"
        )
        doc = Document(page_content=content, metadata={"table": table})
        documents.append(doc)
    # Employee-Project join summary (single doc)
    join_query = '''
        SELECT e.id, e.name, e.role, e.department, e.salary, p.id as project_id, p.name as project_name, p.budget
        FROM employees e
        INNER JOIN projects p ON e.id = p.lead_id
    '''
    cursor.execute(join_query)
    rows = cursor.fetchall()
    join_lines = []
    for row in rows:
        emp_id, emp_name, emp_role, emp_dept, emp_salary, proj_id, proj_name, proj_budget = row
        join_lines.append(
            f"Employee: {emp_name} (ID: {emp_id}), Role: {emp_role}, Dept: {emp_dept}, Salary: {emp_salary} | "
            f"Leads Project: {proj_name} (ID: {proj_id}, Budget: {proj_budget})"
        )
    if join_lines:
        join_content = "\n".join(join_lines)
        doc = Document(
            page_content=f"Employee-Project Joins:\n{join_content}",
            metadata={"type": "employee_project_join"}
        )
        documents.append(doc)
    conn.close()
    return documents


In [7]:
db_table_summary_docs("../data/database/company.db")


[Document(metadata={'table': 'employees'}, page_content="Table: employees\nColumns: ['id', 'name', 'role', 'department', 'salary']\nTotal Records: 4\nSample Record (first 5 fields): {'id': 1, 'name': 'Alice Johnson', 'role': 'Software Engineer', 'department': 'Development', 'salary': 90000.0}"),
 Document(metadata={'table': 'projects'}, page_content="Table: projects\nColumns: ['id', 'name', 'budget', 'lead_id']\nTotal Records: 4\nSample Record (first 5 fields): {'id': 1, 'name': 'Project Alpha', 'budget': 150000.0, 'lead_id': 3}"),
 Document(metadata={'type': 'employee_project_join'}, page_content='Employee-Project Joins:\nEmployee: Charlie Brown (ID: 3), Role: Product Manager, Dept: Product, Salary: 105000.0 | Leads Project: Project Alpha (ID: 1, Budget: 150000.0)\nEmployee: Alice Johnson (ID: 1), Role: Software Engineer, Dept: Development, Salary: 90000.0 | Leads Project: Project Beta (ID: 2, Budget: 200000.0)\nEmployee: Bob Smith (ID: 2), Role: Data Scientist, Dept: Data Science, Sa